In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [2]:
df = pd.read_csv('recommand_data.csv',encoding = 'CP949')

In [3]:
pd.set_option('display.max_columns',None)

In [4]:
indices_to_remove = df[(df['학위과정명'] == '박사')|
                        (df['학위과정명'] == '박사+석박사통합')|
                        (df['학위과정명'] == '석박사통합')|
                        (df['학위과정명'] == '석사')|
                        (df['학위과정명'] == '석사+박사')|
                        (df['학위과정명'] == '석사+박사+석박사통합')|
                        (df['학위과정명'] == '석사+석박사통합')|
                        (df['학위과정명'] == '전문기술석사')|
                        (df['학과상태명'] == '폐과')|
                        (df['학교구분명'] == '일반대학원')|
                        (df['학교구분명'] == '전문대학원')|
                        (df['학교구분명'] == '특수대학원')|
                        (df['학교학과특성명'] == '전문기술석사과정')|
                        (df['학교학과특성명'] == '학석사통합과정')
].index
df = df.drop(indices_to_remove)

In [5]:
df=df.drop(['연도', '시도코드', '시군구코드','학과코드명(7대계열)','표준분류계열코드','수정일자','데이터기준일자','학과상태명','시군구명','시도명'
           ,'입학정원수','졸업자수'],axis=1)
df=df.fillna('')

In [6]:
df

,학교명,학교구분명,수업연한,학위과정명,주야과정명,학과명,대학자체계열명,단과대학명,학교학과특성명,주요교과목명,관련직업명
0,ICT폴리텍대학,기능대학,2년,전문학사,주간,AI영상보안학과,공학,단과대구분없음,일반과정,IoT실습+컴퓨터구조+딥러닝영상분석실습+방송통신기기+프로그래밍일반+네트워크구축실습+...,디지털영상처리전문가+방송제작관리자+정보시스템운영자+통신 및 관련 장비설치 및 수리원
1,ICT폴리텍대학,기능대학,2년,전문학사,주간,IT융합학과,공학,단과대구분없음,일반과정,프로젝트실습(III)+5G이동통신시스템실습+빅데이터분석+통신회로실습+현장실습+안테나...,무선설비산업기사+정보보호전문가+정보시스템운영자+통신 및 관련 장비설치 및 수리원+통...
2,ICT폴리텍대학,기능대학,2년,전문학사,주간,기타(소속학과없음),인문사회,단과대구분없음,일반과정,,
5,ICT폴리텍대학,기능대학,2년,전문학사,주간,스마트통신학과,공학,단과대구분없음,일반과정,전자회로+회로이론+정보통신설비법규+스마트통신공학+스마트광통신공학+컴퓨터네트워크+스마...,정보보호전문가+정보시스템운영자+통신 및 관련 장비설치 및 수리원+통신 및 관련 장비...
7,ICT폴리텍대학,기능대학,2년,전문학사,주간,정보보안학과,공학,단과대구분없음,일반과정,침해대응실습+네트워크일반+네트워크분석실습+컴퓨터구조+네트워크구축실습+JAVA언어실습...,정보보호전문가+컴퓨터보안전문가+통신 및 관련 장비설치 및 수리원
...,...,...,...,...,...,...,...,...,...,...,...
54301,화신사이버대학교,사이버대학(대학),4년,학사,원격,스마트팜농수산복지학과,자연과학,단과대구분없음,일반과정,가족복지론+과수원예학+노인복지론+농업학개론+농촌복지론+발효식품학+사회문제론+사회복지...,귀농귀촌플래너+농업기술자+농촌지도사+도시농업연구원+사회복지사+임상심리사(심리치료사)...
54303,화신사이버대학교,사이버대학(대학),4년,학사,원격,실용외국어학과,인문사회,단과대구분없음,일반과정,TOEIC L/C+TOEIC R/C+국제이주와 사회통합+국제이주와 젠더+국제이주와노...,관광통역안내원+방과후교사+번역가+여행안내원+외교관+외국어교사+외국어학원강사+통역가
54305,화신사이버대학교,사이버대학(대학),4년,학사,원격,영상콘텐츠학과,예체능,단과대구분없음,일반과정,1인미디어제작+국제이주와 노동정책+국제이주와 사회통합+국제이주와 젠더+다문화가족상담...,게임그래픽디자이너+디지털영상처리전문가+멀티미디어콘텐츠제작전문가+방송카메라 기자+영상...
54306,화신사이버대학교,사이버대학(대학),4년,학사,원격,융합스포츠지도학과,예체능,단과대구분없음,일반과정,국제이주와 노동정책+국제이주와 사회통합+국제이주와 젠더+다문화가족상담이론과 실제+다...,각종스포츠지도자+다문화사회지도사+생활체육지도자+스포츠강사+청소년지도사


In [26]:
df['주요교과목명'] = df['주요교과목명'].fillna('')

In [28]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['주요교과목명'])

In [64]:
knn = NearestNeighbors(n_neighbors=15245, metric='cosine').fit(tfidf_matrix)  # n_neighbors를 제거하여 모든 이웃을 고려

In [65]:
def recommend_major(major_subject, n_recommendations = 15245  ):
    input_vector = tfidf_vectorizer.transform([major_subject]).toarray()
    
    distances, indices = knn.kneighbors(input_vector, n_neighbors=n_recommendations)
    
    recommendations = df.iloc[indices[0]].copy()
    recommendations['distance'] = distances[0]

    recommendations = recommendations.sort_values(by='distance')
    
    return recommendations 

In [96]:
input_major_subject = "프로그래밍"

In [97]:
recommendations = recommend_major(input_major_subject)

In [98]:
recommendations = recommendations.drop(['대학자체계열명_encoded'],axis=1)

In [99]:
import re
def replace_text(text):
    # 정규식 패턴: '+'로 연결된 부분 중 'C++'가 포함되지 않은 경우에만 매칭
    pattern = r'\b(?!C\+\+)\w+(\s*\+\s*\w+)*\b'
    
    def replacer(match):
        # 매칭된 부분에서 '+'를 ','로 교체
        return match.group(0).replace('+', ', ')

    # 정규식 패턴에 매칭되는 부분만 교체
    result = re.sub(pattern, replacer, text)
    return result

In [100]:
recommendations['주요교과목명'] = recommendations['주요교과목명'].apply(lambda x: replace_text(x))
recommendations['관련직업명'] = recommendations['관련직업명'].apply(lambda x: replace_text(x))

In [101]:
recommendations

,학교명,학교구분명,수업연한,학위과정명,주야과정명,학과명,대학자체계열명,단과대학명,학교학과특성명,주요교과목명,관련직업명,distance
21058,동서울대학교,전문대학,2년,전문학사,주간,AI응용소프트웨어과,공학,단과대구분없음,계약학과,"IoT 프로그래밍, 소프트웨어공학 응용, 객체지향 프로그래밍, 웹 어플리케이션 구축...","개인미디어콘텐츠제작자, 게임기획자, 시스템소프트웨어개발자",0.475448
51091,한서대학교,대학교,4년,학사,주간,항공컴퓨터전공,공학,Hans 국제항공대학,일반과정,"C 프로그래밍 , IOT 기반 자동제어, 가상현실과 공항관리, 기계학습 개론, 논리...","연구원, 항공공학기술자",0.496571
23285,명지대학교,대학교,4년,학사,주간,융합소프트웨어학부,공학,ICT융합대학,일반과정,"DB설계 및 구현1, DB설계 및 구현2, R통계분석실습, SW기술세미나, SW기술...","데이터베이스개발자, 데이터베이스관리자, 스마트폰앱개발자, 정보기술컨설턴트, 정보시스...",0.543174
23642,명지전문대학,전문대학,3년,전문학사,주간,정보통신공학과,공학,단과대구분없음,일반과정,"벤처 창업, 특허실무, C 언어, IoT, 객체지향 언어 설계 , 객체지향언어실습,...","RFID시스템개발자, 가상현실전문가, 게임기획자, 게임프로그래머, 네트워크프로그래머...",0.586919
38837,유원대학교,대학교,4년,학사,주간,스마트IT전공,공학,단과대구분없음,일반과정,"창의적 종합설계Ⅱ, 프론트엔드 프로그래밍, 장기현장실습Ⅰ, 장기현장실습Ⅲ, 리눅스 ...","가상현실전문가, 게임프로그래머",0.596648
...,...,...,...,...,...,...,...,...,...,...,...,...
53183,호서대학교,대학교,4년,학사,주간,사회복지학부,인문사회,인문사회대학,일반과정,[노인복지학트랙]AI와고령친화사업+[노인복지학트랙]가족상담및치료+[노인복지학트랙]건...,"건강가정사, 공무원, 노인상담복지전문가, 다문화사회지도사, 사회복지관련관리자, 사회...",1.000000
53191,호서대학교,대학교,4년,학사,주간,사회체육학과,예체능,예체능대학,일반과정,"체육원리, 태권도1, 스포츠경영학, 테니스1, 에어로빅댄스1, 축구, 운동역학, 운...","각종스포츠지도자, 생활체육지도자, 선수트레이너, 스포츠강사, 스포츠과학연구원, 퍼스...",1.000000
53193,호서대학교,대학교,4년,학사,주간,산업디자인학과,예체능,예체능대학,일반과정,"기초디자인1, 디자인제도, 기초표현기법2, 기초표현기법1, 기초디자인2, 현대디자인...",제품디자이너,1.000000
53679,호원대학교,산업대학,4년,학사,야간,사회복지학과,인문사회,사회안전대학,산업체위탁,"가족상담및가족치료, 노인복지론, 사회문제론, 사회복지법제와실천, 사회복지실천기술론,...","공무원, 복지시설생활지도원, 사회복지관련관리자, 사회복지사",1.000000


In [69]:
import joblib

# 모델 저장
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')
joblib.dump(knn, 'knn_model.pkl')

['knn_model.pkl']